In [1]:
import PyPDF2
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

results = []
with open('meeting/회의록.pdf', "rb") as f:
    pdf_reader = PyPDF2.PdfReader(f)
    for page_num in range(len(pdf_reader.pages)) :
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        print(f"Page : {page_num} / Text: {text} \n")
        results.append(f"Page : {page_num} / Text: {text} \n")



Page : 0 / Text: 참석자 1 03:44
한 분 더 들어오시는 거죠?
참석자 2 03:47
네네 지금
참석자 1 03:48
여쭙고 있습니다.
참석자 1 04:32
아
참석자 1 04:38
네 그러면 다 들어오신 것 같으니까 시작할까요? 네네 정우 님이 그러면 이제 
준비하신 거 말씀해 주세요.
참석자 2 04:50
네 알겠습니다. 화면 공유해 드릴게요. 네 혹시 보이시나요?
네 잘 보입니다. 네네. 일단은 마지막 멘토링 때 멘토님께서 저희에게 요청하셨
던 내용들 한번 간단하게 정리한 다음에 바로 보여드릴게요.
대부분의 내용이 요구 사항 기능서랑 프로젝트 기획서 수정 방안이라고 저희가 
판단을 했고 그다음에 나머지 부분들은 시장 조사 같은 경우에는 자체 LLM 방
향성보다는 사내 문서 검색 시스템 쪽으로 시장 조사를 좀 더 추가하고 수정해
라 이렇게 저희가 생각을 했고 그리고
참석자 2 05:37
나머지 부분은 다 요구 사항 기능서랑 프로젝트 기획서에 대한 내용으로 알고  

Page : 1 / Text: 있습니다.
참석자 1 05:45
지금 이게 제가 그때 말씀드린 거죠. 그러면 이것들 하나씩 한번 일단 먼저 볼
까요?
참석자 2 05:51
네 알겠습니다. 그 가상의 페르소나를 만들어서 이야기를 풀어가는 스토리텔링 
식으로 만 작성하면 좋겠다라고 말씀해 주셔서 저희 팀 힌톤을 딴 신입 사원 분
석가라고 해서 해당 직원이 어떤 문제가 있고 어떤 상황에서 저희 서비스를 쓸 
건지 대략의 상황을 가정해서 일단 적어봤습니다.
참석자 1 06:19
깔끔한데 일단 잘하셨어요. 그리고 문서가 너무 많아 이런 것들은 좀 더 이제 
정돈을 해야 될 것 같은데 그러니까 현재 어떤 문서를 쓸 건지 예를 들면 프로
덕트인지 그런 것들을 정리를 했잖아요.
네 그런 거에 대한 시나리오를 적는 게 좋을 것 같거든요.
그러니까 제가 제품을 그때그때마다 찾으려면 어떤 특정 경로에 들어가가지고 
문서를 뒤졌어야 되는데
참석자 2 06:49
네네
참석자 1 06:50
그러니까 거기도

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import load_dotenv
from IPython.display import Markdown, display
load_dotenv()

# 1. 메시지 템플릿 생성
system_message = SystemMessagePromptTemplate.from_template(
    "너는 어시스턴트야. 내가 회의록 PDF 파일을 주면, 첫번째로 회의 주제 / 목적, "
    "두번째로 핵심 논의 사항, 세번째로 결정된 사항, 네번째로 보류되거나 추가 논의 항목을 "
    "요약해서 한국어로 알려줘."
)
human_message = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# 2. LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. Prompt와 LLM 결합
chatbot = chat_prompt | llm

# 4. 실행
response = chatbot.invoke({"input": results})
result = response.content
print(result)

### 회의 주제 / 목적
이번 회의는 프로젝트 기획서와 요구 사항 기능서 수정을 논의하고, 시장 조사 및 기술적 방향성에 대한 피드백을 받으며, 각 팀원들의 역할 분배에 대해 논의하기 위한 자리였습니다.

### 핵심 논의 사항
1. **요구 사항 기능서와 프로젝트 기획서**: 참석자들은 멘토의 피드백을 바탕으로 수정할 내용을 논의했으며, 요구 사항 기능서에서 데이터 처리 및 에이전트 기능 통합을 추가하는 방향으로 수정을 결정했습니다.
2. **시장 조사**: 시장 조사 내용에 대한 논의가 있었으며, 사내 문서 검색 시스템의 필요성과 이를 통한 효율성을 강조했습니다.
3. **기술 스펙 및 모델링 방안**: 프로젝트에서 사용될 기술 스펙을 정리하고, 모델링 방안에 대해 구체적인 의견을 교환했습니다.

### 결정된 사항
1. 요구 사항 기능서와 프로젝트 기획서를 수정하여 멘토의 피드백을 반영하기로 하였습니다.
2. 각 팀원이 맡을 역할을 명확히 하고, 필요 시 추가적인 스터디를 진행하기로 하였습니다.
3. 시장 조사 결과를 바탕으로 사내 문서 검색 시스템 구축 방향성을 구체화하기로 하였습니다.

### 보류되거나 추가 논의 항목
1. **시장 조사 내용**: 경쟁사 분석 및 특정 기술 동향에 대한 추가 조사가 필요합니다.
2. **기술적 논의**: 에이전트 구성 및 시스템 아키텍처에 대한 구체적 세부 사항은 다음 회의에서 추가적으로 논의하기로 하였습니다.
3. **일정 조율**: 팀원 간의 일정 조율 및 다음 회의 형태에 대한 논의가 필요합니다.


In [15]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("meeting/회의록.pdf")
import pdfplumber
with pdfplumber.open("meeting/회의록.pdf") as pdf :
    pages = pdf.pages # 페이지 리스트
    for page in pages : 
        im = page.to_image(resolution=150) # Page 객체 -> PageImage라는 객체로 반환
        im.draw_rects(page.extract_words()) # 페이지에서 추출한 단어들의 위치 좌표에 사각형(rect)을 그리는 함수입니다.

# 마지막 페이지만 화면 그대로 출력
im.show()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def